In [103]:
from pyg.twod import pyg2d, svg_show
from pym import func as pym
from pyg.colors import pu as puc
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import re
pyg2d.change_context('tufte')

# The Design

Any acoustic system is a resonant system.  When coupling the acoustic media of power transfer (air or water) with the piezoelectric transducer and a circuit, the system becomes triply resonant.  The matching of the three resonances (acoustically, piezoelectrically, and electrically) becomes the prime factor for operation of a power harvester.  The process of matching these resonances was carried out iteratively, with the iteration steps being:

1. Choice of PZT material and shape
2. Choice of frequency range for design
    1. Calculation of PZT size from frequency range
    2. Verification of PZT resonances using COMSOL
    3. Verification of PZT resonances using SPICE
    4. Design of resonant circuit for **real** resonances from modeling software
        1. Design of $LC$ circuit to match frequency of resonance
        2. Design of $RC$ circuit to match frequency of resonance
        3. Comparison and choice between $LC$ and $RC$
    5. Simulation of power output in SPICE

## PZT Choice

- Following Botteron, and Illich, using PZT-5A because of its common availability
- This is often called Navy PZT II
- Disk shape chosen

## PZT Size Calculation

- Frequency of resonance chosen at $500\mathrm{kHz}$ for availability of amplifiers

In [124]:
%autoreload 2
svg_show('pzt_diagram.svg', caption='Coordinate system for disk pzt', width='1')

- Planar Mode ($d>5t$)
$$f_{res}=\frac{N_r}{d} \therefore d=\frac{N_r}{f_{res}}$$
$$d=2.9\mathrm{mm}$$
$$t<0.58\mathrm{mm}$$

## Resonant Circuit Design

In [119]:
svg_show('rlc.svg', caption='Diagram of series RLC circuit', width='1')

$$f_{res} = \frac{1}{2\pi \sqrt{LC}}$$

## Resonant Circuit Choice

- The pzt has an internal capacitance of

$$C=\frac{\pi}{4}k_{33}\varepsilon_{0}\frac{d^{2}}{t}=0.1\mathrm{nF}$$

- Ballast the pzt with a larger capacitor to decrease the size required in the inductor (space saving)
- Ballast capacitor of $10\mathrm{\mu F}$, gives inductor size of $10\mathrm{nH}$

## Regulator Choice

- Tried two ultra-low power boost converters - (TI's BQ25504 as per Botteron, LT's LTC3108), low success, details to follow
- Chose a voltage regulator (need to step-**down** the voltage from PZT)
- High speed required to match $500\mathrm{kHz}$
- Output of $3.6\mathrm{V}$ chosen to match a single cell LiPo battery
- Load on back end of $100\mathrm{k\Omega}$ matches common microprocessors

In [118]:
svg_show('receiver_design.svg', caption='System design before iteration', width='1')

# The Simulation

- Complex components to design
- Acoustic transport to PZT face
- Constitutive equation solution inside PZT Material (including elastic stress/strain)
- Electrical solution in frequency space (Kirchoff's Laws, complicated by the IC)
- Tools possible - COMSOL (FEM solver), SPICE (circuit solver)

## COMSOL Multiphysics Simulation

In [121]:
svg_show('pzt_comsol_mesh.svg',
         caption='Mesh and boundary setup for finite element simulation of PZT-5A disk',
         width='1')

## COMSOL Boundary condition

- Particle Velocity

$$v=1.0\times 10^{-6}\mathrm{\frac{m}{s}} \equiv 20\mathrm{dB}$$

- Conversion to acceleration

$$a = \frac{d}{dt}v=\frac{d}{dt} \left[A \sin \left(2 \pi f t\right)\right]$$
$$a = 2 \pi f A$$

## COMSOL Voltage Results

In [19]:
# plot voltage result

frequencies = []
vpwrout = []
vpztin = []
ipwrout = []
ipztin = []
with open('power_harvester_with_lt1585_freq_space.txt') as f:
    f.next()
    for line in f:
        string = re.sub(r'[^\w\s\n0-9a-zA-Z+-.]+', '*', line)
        string = string.replace('(', '').replace(')', '').replace('dB', '').replace('*', '').replace(',', ' ')
        arr = string.split()
        frequencies.extend([float(arr[0])])
        vpwrout.extend([float(arr[1])])
        vpztin.extend([float(arr[3])])
        ipwrout.extend([float(arr[5])])
        ipztin.extend([float(arr[7])])

        
vpwrout = np.array(vpwrout)
vpztin = np.array(vpztin)
ipwrout = np.array(ipwrout)
ipztin = np.array(ipztin)
vpwroutc = pym.curve(frequencies, np.power(10.0, (vpwrout/20.0)) * 1.0e-6, name='spice vpwrout')
ipwroutc = pym.curve(frequencies, np.power(10.0, (ipwrout/20.0)) * 1.0e-6, name='spice ipwrout')
vpztinc = pym.curve(frequencies, np.power(10.0, (vpztin/20.0)) * 1.0e-6,
                    name='SPICE $V$')
ipztinc = pym.curve(frequencies, np.power(10.0, (ipztin/20.0)) * 1.0e-6, name='SPICE $I$')

(freq, vel, intvolt, intcurr, maxvolt) = np.loadtxt('pzt_power_harvester.csv', unpack=True, skiprows=5, delimiter=',')

freq = freq
intvolt = np.abs(intvolt)

intvoltc = pym.curve(freq, intvolt / 0.0029, name=r'COMSOL $\nicefrac{\int V dx}{d}$')
plot = intvoltc.plot(linestyle='-', linecolor=puc.pu_colors['newgold'])
plot = vpztinc.plot(linestyle='--', linecolor=puc.pu_colors['gray'], addto=plot, yy=True)
plot.markers_off()
plot.lines_on()
plot.semi_log_x()
plot.semi_log_y()
plot.semi_log_y(axes=plot.ax2)
plot.legend(loc=2)
plot.legend(loc=1, axes=plot.ax2)
plot.xlabel('Frequency ($f$) [$\mathrm{Hz}]$')
plot.ylabel(r'Voltage ($\frac{\int{V dx}}{d}$) [$\mathrm{V}$]')
plot.ylabel('Voltage ($V$) [$\unit{V}$]', axes=plot.ax2)
plot.export('intvolt', sizes=['2'], ratio='silver')
plot.show(caption='Across capacitor voltage versus frequency')

## COMSOL Current

In [20]:
# plot current result
intcurrc = pym.curve(freq, intcurr, name=r'COMSOL $\nicefrac{\int{\frac{I}{V} dx}}{d}$')
plot = intcurrc.plot(linestyle='-', linecolor=puc.pu_colors['teal'])
plot = ipztinc.plot(linestyle='--', linecolor=puc.pu_colors['blue'], addto=plot, yy=True)
plot.markers_off()
plot.lines_on()
plot.semi_log_x()
plot.semi_log_y()
plot.semi_log_y(axes=plot.ax2)
plot.legend(loc=2)
plot.legend(loc=1, axes=plot.ax2)
plot.xlabel(r'Frequency ($f$) [$\mathrm{Hz}]$')
plot.ylabel(r'Current Density ($\nicefrac{\int{\frac{I}{V} dx}}{d}$) [$\unit{\nicefrac{A}{m^{3}}}$]')
plot.ylabel(r'Current ($I$) [$\unit{A}$]', axes=plot.ax2)
plot.export('intcurr', sizes=['2'], ratio='silver')
plot.show(caption='Current density generated in piezoelectric harvester versus frequency')

In [21]:
intpow = intcurrc * intvoltc
pztpow = ipztinc * vpztinc
plot = intpow.plot(linestyle='-', linecolor=puc.pu_colors['purple'])
plot = pztpow.plot(linestyle='--', linecolor=puc.pu_colors['red'], addto=plot, yy=True)
plot.markers_off()
plot.lines_on()
plot.semi_log_x()
plot.semi_log_y()
plot.semi_log_y(axes=plot.ax2)
plot.add_data_pointer(pztpow.find_max(), curve=pztpow,
                      string=r'$\underset{f}{\arg\max} \left|' +
                             r' P_{spice}\left( f \right) \right| = %.2e \unit{Hz}$' % pztpow.find_max(),
                      place=(1.0e4, 1.0e-20), axes=plot.ax2)
plot.legend(loc=2)
plot.legend(loc=1, axes=plot.ax2)
plot.xlabel(r'Frequency ($f$) [$\unit{kHz}]$')
plot.ylabel(r'Power ($\int P dx$) [$\unit{W}$]')
plot.ylabel(r'Power ($P$) [$\unit{W}$]', axes=plot.ax2)
plot.export('intpow', sizes=['2'], ratio='silver')
plot.show(caption='Power generated in PZT with frequency')

## LTSpice Model

- Illich created
    - a PZT spice component
    - a finite acoustic media component
    - an infinite acoustic media component
- Illich assumed a current $\Leftrightarrow$ particle velocity analogy from Schwarz
- Allows for easy modeling of circuit using pzt component


## Failed LTSpice Models

In [24]:
svg_show('power_harvester_with_transformer.svg',
         caption='Circuit diagram for circuit iteration using transformer and LT3108', width='2')

- Maximum voltage of $\mathrm{\mu V}$
- Maximum current of $\mathrm{n A}$

## LTSpice Model

In [51]:
svg_show('power_harvester_with_lt1585_lrc.svg',
         caption='Circuit diagram for circuit iteration using LRC oscilating circuit and LT1587-3.3', width='2')

## LTSpice Model Results

In [28]:
# Plot current and voltage from LTSpice freq sweep
fs = []
data = []
prevline = '0.0'
ts = []
vs = []
iis = []
all_fs = []
all_ts = []
all_vs = []
vcurves = {}
with open('power_harvester_with_lt1585_freq_sweep.txt') as f:
    f.next()
    for line in f:
        try:
            arr = line.split()
            t = float(arr[0])
            v = float(arr[1])
            i = float(arr[2])
            all_fs.extend([float(prevline)])
            all_ts.extend([t])
            all_vs.extend([v])
            ts.extend([t])
            vs.extend([v])
            iis.extend([i])
        except (ValueError, IndexError):
            if 'Freq=' in line:
                string = re.sub('\(Run: [0-9]/[0-9]\)', '', line)
                string = string.replace('Step Information: Freq=', '').replace('K', 'e3').replace('M', 'e6')
                fs.extend([float(prevline)])
                data.extend([[np.array(ts), np.array(vs), np.array(iis)]])
                vcurves[float(prevline)] = pym.curve(ts, vs, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                prevline=string
                ts = []
                vs = []
                iis = []
all_fs = np.array(all_fs)
all_ts = np.array(all_ts)
all_vs = np.array(all_vs)
min_t = np.min(all_ts)
min_f = np.min(all_fs)
min_v = np.min(all_vs)
max_t = np.max(all_ts)
max_f = np.max(all_fs)
max_v = np.max(all_vs)
del vcurves[0]

X, Y = np.meshgrid(sorted(vcurves.keys()), np.linspace(min_t, max_t, 100))
Z = np.zeros_like(X)
for i, key in enumerate(sorted(vcurves.keys())):
    cur = vcurves[key]
    for _x, j in zip(np.linspace(min_t, max_t, 100), range(100)):
        Z[j, i] = cur.at(_x)

In [37]:
from pyg.threed import pyg3d
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.array(sorted(vcurves.keys())) / 1.0e3, 1.0e6 * np.linspace(min_t, max_t, 100), Z, cmap=puc.flame_cmap,
          alpha=0.8, addto=plot)
plot.zlabel(r'Voltage ($V$) [$\unit{V}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Frequency ($f$) [$\mathrm{kHz}$]')
plot.export('volt3d', ratio='silver')
plot.show('Voltage at power out terminal versus time versus frequency')

In [38]:
# Plot current and voltage from LTSpice freq sweep
fs = []
data = []
prevline = '0.0'
ts = []
vs = []
iis = []
all_fs = []
all_ts = []
all_vs = []
vcurves = {}
with open('power_harvester_with_lt1585_freq_sweep.txt') as f:
    f.next()
    for line in f:
        try:
            arr = line.split()
            t = float(arr[0])
            v = float(arr[1])
            i = float(arr[2])
            all_fs.extend([float(prevline)])
            all_ts.extend([t])
            all_vs.extend([i])
            ts.extend([t])
            vs.extend([v])
            iis.extend([i])
        except (ValueError, IndexError):
            if 'Freq=' in line:
                string = re.sub('\(Run: [0-9]/[0-9]\)', '', line)
                string = string.replace('Step Information: Freq=', '').replace('K', 'e3').replace('M', 'e6')
                fs.extend([float(prevline)])
                data.extend([[np.array(ts), np.array(vs), np.array(iis)]])
                vcurves[float(prevline)] = pym.curve(ts, iis, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                prevline=string
                ts = []
                vs = []
                iis = []
all_fs = np.array(all_fs)
all_ts = np.array(all_ts)
all_vs = np.array(all_vs)
min_t = np.min(all_ts)
min_f = np.min(all_fs)
min_v = np.min(all_vs)
max_t = np.max(all_ts)
max_f = np.max(all_fs)
max_v = np.max(all_vs)
del vcurves[0]

X, Y = np.meshgrid(sorted(vcurves.keys()), np.linspace(min_t, max_t, 100))
Z = np.zeros_like(X)
for i, key in enumerate(sorted(vcurves.keys())):
    cur = vcurves[key]
    for _x, j in zip(np.linspace(min_t, max_t, 100), range(100)):
        Z[j, i] = cur.at(_x)

In [39]:
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.array(sorted(vcurves.keys())) / 1.0e3, 1.0e6 * np.linspace(min_t, max_t, 100), 1.0e3 * Z,
          cmap=puc.flow_cmap,
          alpha=0.8, addto=plot)
plot.zlabel(r'Current ($I$) [$\unit{mA}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Frequency ($f$) [$\mathrm{kHz}$]')
plot.export('curr3d', ratio='silver')
plot.show('Current at power out terminal versus time versus frequency')

In [46]:
# Plot current and voltage from LTSpice sndamp sweep
(t, vpwrout, vpztin, ipwrout, ipztin) = np.loadtxt('power_harvester_with_lt1585.txt', unpack=True, skiprows=1)
(trc, vpwroutrc, ipwroutrc) = \
    np.loadtxt('power_harvester_with_lt1585_rc.txt', unpack=True, skiprows=1)

vpwroutc = pym.curve(1.0E6 * t, vpwrout, name='$V_{lc}$')
vpwroutrcc = pym.curve(1.0E6 * trc, vpwroutrc, name='$V_{rc}$')
plot = vpwroutc.plot(linestyle='-', linecolor=puc.pu_colors['newgold'])
plot = vpwroutrcc.plot(linestyle='--', linecolor=puc.pu_colors['oldgold'], addto=plot)
plot.markers_off()
plot.lines_on()
plot.legend()
plot.xlabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.ylabel('Voltage ($V$) [$\mathrm{V}$]')
plot.xlim(0, 4)
plot.ylim(0, 4.5)
plot.export('volt', ratio='silver')
plot.show('Voltage at power out terminal versus time')

## LTSpice Model Results

In [47]:
ipwroutc = pym.curve(1.0E6 * t, 1.0E6 * ipwrout, name='$I_{lc}$')
ipwroutrcc = pym.curve(1.0E6 * trc, 1.0E6 * ipwroutrc, name='$I_{rc}$')
plot = ipwroutc.plot(linestyle='-', linecolor=puc.pu_colors['teal'])
plot = ipwroutrcc.plot(linestyle='--', linecolor=puc.pu_colors['blue'], addto=plot)
plot.markers_off()
plot.lines_on()
plot.legend()
plot.xlabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.ylabel('Current ($I$) [$\mathrm{\mu A}$]')
plot.xlim(0, 4)
plot.ylim(0, 50)
plot.export('curr', ratio='silver')
plot.show('Current at power out terminal versus time')

In [50]:
ppwroutc = ipwroutc * vpwroutc
ppwroutc.name = '$P_{lc}$'
ppwroutrcc = ipwroutrcc * vpwroutrcc
ppwroutrcc.name = '$P_{rc}$'
plot = ppwroutc.plot(linestyle='-', linecolor=puc.pu_colors['purple'])
plot = ppwroutrcc.plot(linestyle='--', linecolor=puc.pu_colors['red'], addto=plot)
plot.markers_off()
plot.lines_on()
plot.legend()
plot.xlabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.ylabel('Power ($P$) [$\mathrm{\mu W}$]')
plot.xlim(0, 4)
plot.ylim(0, 165)
plot.export('power', ratio='silver')
plot.show('Power at power out terminal versus time')

# The Realization

- Drop in replacement for single cell LiPo battery ($>3.7\mathrm{V}$)
- After start up period of $150\mathrm{\mu s}$, reaches steady state current of $42\mathrm{\mu A}$
- Provides power of $176.4\mathrm{\mu W}$
- Size of $6.6\mathrm{mm^{2}}$

# The Future

- Optimization studies into LC circuit values
- Better PZT material possibilities
- More efficient step-down conversion
- Higher current from higher frequencies ($5\mathrm{MHz}$)
- Design of PCB for size estimates

In [108]:
# Plot current and voltage from LTSpice freq sweep
fs = []
data = []
prevline = '0.0'
ts = []
vs = []
iis = []
all_fs = []
all_ts = []
all_vs = []
vcurves = {}
icurves = {}
with open('power_harvester_with_lt1585_lrc_r.txt') as f:
    f.next()
    for line in f:
        try:
            arr = line.split()
            t = float(arr[0])
            v = float(arr[1])
            i = float(arr[2])
            all_fs.extend([float(prevline)])
            all_ts.extend([t])
            all_vs.extend([v])
            ts.extend([t])
            vs.extend([v])
            iis.extend([i])
        except (ValueError, IndexError):
            if 'Rmatch=' in line:
                string = line
                string = re.sub('\(Run: [0-9]+/[0-9]+\)', '', string)
                string = string.replace('Step Information: Rmatch=', '')\
                    .replace('K', 'e3').replace('M', 'e6').replace('m', 'e-3').replace('n', 'e-9')\
                    .replace('nan', 'np.nan')
                string = re.sub('[^\w\s\(\)a-zA-Z0-9.+-:/]', 'e-6', string)
                fs.extend([float(prevline)])
                data.extend([[np.array(ts), np.array(vs), np.array(iis)]])
                vcurves[float(prevline)] = pym.curve(ts, vs, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                icurves[float(prevline)] = pym.curve(ts, iis, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                prevline=string
                ts = []
                vs = []
                iis = []
all_fs = np.array(all_fs)
all_ts = np.array(all_ts)
all_vs = np.array(all_vs)
min_t = np.min(all_ts)
min_f = np.min(all_fs)
min_v = np.min(all_vs)
max_t = np.max(all_ts)
max_f = np.max(all_fs)
max_v = np.max(all_vs)
del vcurves[0]
del icurves[0]

X, Y = np.meshgrid(sorted(vcurves.keys()), np.linspace(min_t, max_t, 100))
Z = np.zeros_like(X)
A = np.zeros_like(X)
for i, key in enumerate(sorted(vcurves.keys())):
    cur = vcurves[key]
    icur = icurves[key]
    for _x, j in zip(np.linspace(min_t, max_t, 100), range(100)):
        Z[j, i] = cur.at(_x)
        A[j, i] = icur.at(_x)

In [111]:
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.log10(np.array(sorted(vcurves.keys()))), 1.0e6 * np.linspace(min_t, max_t, 100), Z, cmap=puc.flame_cmap,
          alpha=0.8, addto=plot)
plot.zlabel(r'Voltage ($V$) [$\unit{V}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Resistance ($R$) [$\Omega$]')
plot.ax.xaxis.set_major_formatter(FormatStrFormatter('$10^{%.0f}$'))
plot.export('volt3d_res', ratio='silver')
plot.show('Voltage at power out terminal versus time versus matching resistance')

In [112]:
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.log10(np.array(sorted(vcurves.keys()))), 1.0e6 * np.linspace(min_t, max_t, 100), 1.0e6 * A, cmap=puc.flow_cmap,
          alpha=0.8, addto=plot)
plot.zlabel(r'Current ($I$) [$\unit{\mu A}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Resistance ($R$) [$\Omega$]')
plot.ax.xaxis.set_major_formatter(FormatStrFormatter('$10^{%.0f}$'))
plot.export('curr3d_res', ratio='silver')
plot.show('Current at power out terminal versus time versus matching resistance')

In [113]:
# Plot current and voltage from LTSpice freq sweep
fs = []
data = []
prevline = '0.0'
ts = []
vs = []
iis = []
all_fs = []
all_ts = []
all_vs = []
vcurves = {}
icurves = {}
with open('power_harvester_with_lt1585_lrc_lmatch.txt') as f:
    f.next()
    for line in f:
        try:
            arr = line.split()
            t = float(arr[0])
            v = float(arr[1])
            i = float(arr[2])
            all_fs.extend([float(prevline)])
            all_ts.extend([t])
            all_vs.extend([v])
            ts.extend([t])
            vs.extend([v])
            iis.extend([i])
        except (ValueError, IndexError):
            if 'Lmatch=' in line:
                string = re.sub('\(Run: [0-9]+/[0-9]+\)', '', line)
                string = string.replace('Step Information: Lmatch=', '')\
                    .replace('K', 'e3').replace('M', 'e6').replace('m', 'e-3').replace('n', 'e-9')\
                    .replace('nan', 'np.nan')
                string = re.sub('[^\w\s\(\)a-zA-Z0-9.+-:/]', 'e-6', string)
                fs.extend([float(prevline)])
                data.extend([[np.array(ts), np.array(vs), np.array(iis)]])
                vcurves[float(prevline)] = pym.curve(ts, vs, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                icurves[float(prevline)] = pym.curve(ts, iis, name=r'$f=%.2e \mathrm{Hz}$' % float(prevline))
                prevline=string
                ts = []
                vs = []
                iis = []
all_fs = np.array(all_fs)
all_ts = np.array(all_ts)
all_vs = np.array(all_vs)
min_t = np.min(all_ts)
min_f = np.min(all_fs)
min_v = np.min(all_vs)
max_t = np.max(all_ts)
max_f = np.max(all_fs)
max_v = np.max(all_vs)
del vcurves[0]
del icurves[0]

X, Y = np.meshgrid(sorted(vcurves.keys()), np.linspace(min_t, max_t, 100))
Z = np.zeros_like(X)
A = np.zeros_like(X)
for i, key in enumerate(sorted(vcurves.keys())):
    cur = vcurves[key]
    icur = icurves[key]
    for _x, j in zip(np.linspace(min_t, max_t, 100), range(100)):
        Z[j, i] = cur.at(_x)
        A[j, i] = icur.at(_x)

In [114]:
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.log10(np.array(sorted(vcurves.keys()))), 1.0e6 * np.linspace(min_t, max_t, 100), Z, cmap=puc.flame_cmap,
          alpha=1.0, addto=plot)
plot.zlabel(r'Voltage ($V$) [$\unit{V}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Inductance ($L$) [$\mathrm{H}$]')
plot.ax.xaxis.set_major_formatter(FormatStrFormatter('$10^{%.0f}$'))
plot.export('volt3d_ind', ratio='silver')
plot.show('Voltage at power out terminal versus time versus matching inductance')

In [115]:
# recoil energy deposited in fluid vs. pneg vs. E
plot = pyg3d()
plot.surf(np.log10(np.array(sorted(vcurves.keys()))), 1.0e6 * np.linspace(min_t, max_t, 100), 1.0e6 * A,
          cmap=puc.flow_cmap,
          alpha=1.0, addto=plot)
plot.zlabel(r'Current ($I$) [$\unit{\mu A}$]')
plot.ylabel('Time ($t$) [$\mathrm{\mu s}]$')
plot.xlabel('Inductance ($L$) [$\mathrm{L}$]')
plot.ax.xaxis.set_major_formatter(FormatStrFormatter('$10^{%.0f}$'))
plot.export('curr3d_ind', ratio='silver')
plot.show('Current at power out terminal versus time versus matching inductance')